In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = 'C:/Users/EunJin/Desktop/dacon/LG AI/'

In [2]:
def make_df(train_err, quality):
  #train_err  = pd.read_csv(PATH+'train_err_data.csv')
  train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
  train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
  train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

  train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
  train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
  train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
  train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))

  ###err_type
  pivot = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
  df_errtype = pd.DataFrame(pivot.to_records())

  df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 

  ###err_day
  pivot2 = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
  df_errday = pd.DataFrame(pivot2.to_records())

  df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

  for i in list(range(1,33)) :
          df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

  df_result = pd.merge(df_errtype, df_errday, on='user_id', how='inner')

  ###quality
  #quality = pd.read_csv(PATH+'train_quality_data.csv')
  quality.fillna(0) #quality_5변수에 NA값이 있음



  #quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
  cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
          'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  quality[cols] = quality[cols].replace({',':''}, regex= True) 
  quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

  #quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

  #minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
  #현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
  minusQ = []
  for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
    if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
    #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
      minusQ.append(-1)
    else:
      minusQ.append(0)

  quality['minusQ'] = minusQ #minusQ 열 추가해주기



  #규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
  #minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

  def func(x):
      d = {}
      d['quality_0'] = x['quality_0'].max()
      d['quality_1'] = x['quality_1'].max()
      d['quality_2'] = x['quality_2'].max()
      d['quality_3'] = x['quality_3'].max()
      d['quality_4'] = x['quality_4'].max()
      d['quality_5'] = x['quality_5'].max()
      d['quality_6'] = x['quality_6'].max()
      d['quality_7'] = x['quality_7'].max()
      d['quality_8'] = x['quality_8'].max()
      d['quality_9'] = x['quality_9'].max()
      d['quality_10'] = x['quality_10'].max()
      d['quality_11'] = x['quality_11'].max()
      d['quality_12'] = x['quality_12'].max()
      d['minusQ'] = x['minusQ'].sum()
      return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                  'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

  quality5 = quality.groupby(['user_id','time']).apply(func)

  quality5 = quality5.groupby(['user_id']).sum()
  quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

  df_result = pd.merge(df_result, quality5, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)

  ###errcode
  pivot3 = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
  df_errcode = pd.DataFrame(pivot3.to_records())

  df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))

  df_result = pd.merge(df_result, df_errcode, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)

  return df_result

In [3]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_quality = pd.read_csv(PATH+'test_quality_data.csv')
X_test = make_df(test_err, test_quality)

KeyboardInterrupt: 

In [4]:
X_test.head()

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,day1203,day1204,day1205,day1209,day1211,day1214,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-1010,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10020,errcode_10029,errcode_10030,errcode_10035,errcode_10043,errcode_10046,errcode_10072,errcode_101,errcode_10158,errcode_10165,errcode_10168,errcode_102,errcode_10217,errcode_10237,errcode_10274,errcode_103,errcode_103063,errcode_10330,errcode_10331,errcode_10334,errcode_10335,errcode_10361,errcode_104,errcode_10400,errcode_10475,errcode_10487,errcode_104942,errcode_105,errcode_10500,errcode_10512,errcode_10522,errcode_10523,errcode_10527,errcode_10553,errcode_10558,errcode_10563,errcode_10564,errcode_10567,errcode_10576,errcode_10580,errcode_10581,errcode_10585,errcode_10591,errcode_10598,errcode_10599,errcode_106,errcode_10609,errcode_10626,errcode_10638,errcode_10651,errcode_10653,errcode_10655,errcode_10659,errcode_10668,errcode_10675,errcode_10680,errcode_10683,errcode_10685,errcode_10686,errcode_10695,errcode_10698,errcode_107,errcode_10711,errcode_10717,errcode_10730,errcode_10731,errcode_10734,errcode_10736,errcode_10743,errcode_10751,errcode_10753,errcode_10763,errcode_10774,errcode_10779,errcode_10784,errcode_10785,errcode_10788,errcode_10790,errcode_108,errcode_10804,errcode_10805,errcode_10806,errcode_10812,errcode_10817,errcode_10822,errcode_10825,errcode_10829,errcode_10830,errcode_10843,errcode_10845,errcode_10848,errcode_10853,errcode_10857,errcode_10860,errcode_10867,errcode_10872,errcode_10874,errcode_108761,errcode_10881,errcode_10890,errcode_10894,errcode_10899,errcode_109,errcode_10911,errcode_10913,errcode_10915,errcode_10920,errcode_10923,errcode_10930,errcode_10935,errcode_10944,errcode_10963,errcode_10966,errcode_10979,errcode_10984,errcode_10985,errcode_109851,errcode_10987,errcode_10991,errcode_10992,errcode_10993,errcode_110,errcode_11002,errcode_11006,errcode_11010,errcode_11011,errcode_11023,errcode_11027,errcode_11037,errcode_11044,errcode_11049,errcode_11067,errcode_11077,errcode_11085,errcode_11090,errcode_11100,errcode_11105,errcode_11116,errcode_11124,errcode_11138,errcode_11160,errcode_11161422,errcode_11226,errcode_11246,errcode_112485,errcode_11308,errcode_11336,errcode_113979,errcode_11398,errcode_11416,...,errcode_8479,errcode_8492,errcode_8497,errcode_85,errcode_8507,errcode_8517,errcode_8540,errcode_8543,errcode_8549,errcode_8554,errcode_8558,errcode_8562,errcode_8564,errcode_8565,errcode_8571,errcode_8572,errcode_8577,errcode_8579,errcode_8583,errcode_8595,errcode_8599,errcode_86,errcode_8603,errcode_8605,errcode_8622,errcode_8630,errcode_8642,errcode_8646,errcode_8663,errcode_8667,errcode_8671,errcode_8682,errcode_8683,errcode_8699,errcode_87,errcode_8708,errcode_8715,errcode_8718,errcode_8720,errcode_8723,errcode_8726,errcode_8730,errcode_8731,errcode_8737,errcode_87429,errcode_8743,errcode_8745,errcode_8754,errcode_8758,errcode_87621,errcode_8763,errcode_8771,errcode_8773,errcode_8784,errcode_88,errcode_8802,errcode_8831,errcode_8834,errcode_8839,errcode_8843,errcode_8848,errcode_8849,errcode_8852,errcode_8855,errcode_8857,errcode_8868,errcode_8881,errcode_8889,errcode_8899,errcode_89,errcode_8913,errcode_8916,errcode_8924,errcode_8932,errcode_8941,errcode_8942,

In [5]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
X_train = make_df(train_err, train_quality)

In [6]:
X_train.head()

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10005,errcode_10018,errcode_10043,errcode_10073,errcode_10080,errcode_101,errcode_10101,errcode_10149,errcode_10153,errcode_10155,errcode_10164,errcode_10190,errcode_102,errcode_10222,errcode_10226,errcode_10240,errcode_10247,errcode_10263,errcode_10267,errcode_10274,errcode_102789,errcode_10280,errcode_103,errcode_10305,errcode_10309,errcode_103400,errcode_10350,errcode_10377,errcode_10385,errcode_103924,errcode_104,errcode_10406,errcode_10412,errcode_10420,errcode_10433,errcode_105,errcode_10518,errcode_10552,errcode_10554,errcode_10557,errcode_10566,errcode_10568,errcode_105690,errcode_10573,errcode_10584,errcode_106,errcode_10612,errcode_10630,errcode_10670,errcode_10674,errcode_10676,errcode_10684,errcode_10689,errcode_10692,errcode_107,errcode_10715,errcode_10720,errcode_10721,errcode_10725,errcode_10732,errcode_10743,errcode_10749,errcode_10752,errcode_10760,errcode_10768,errcode_10777,errcode_10782,errcode_10792,errcode_108,errcode_10803,errcode_10807,errcode_10812,errcode_108133,errcode_10814,errcode_10819,errcode_10827,errcode_10830,errcode_10833,errcode_10838,errcode_10839,errcode_10841,errcode_10851,errcode_10852,errcode_10861,errcode_10881,errcode_10894,errcode_10898,errcode_1089908,errcode_109,errcode_10902,errcode_10907,errcode_10908,errcode_10933,errcode_10934,errcode_10938,errcode_10945,errcode_10949,errcode_10955,errcode_10961,errcode_10972,errcode_10980,errcode_10995,errcode_10997,errcode_110,errcode_11007,errcode_11014,errcode_11015,errcode_11033,errcode_11043,errcode_11063,errcode_11068,errcode_11070,errcode_11078,errcode_11087,errcode_11088,errcode_11094,errcode_11100,errcode_11108,errcode_11113,errcode_11146,errcode_11151,errcode_11197,errcode_11251,errcode_11259,errcode_112910,errcode_11295,errcode_113066,errcode_11324,errcode_11355,errcode_11396,errcode_11399,errcode_11434,errcode_11516,errcode_11534,errcode_11650,errcode_1166733,errcode_11674,errcode_11717,errcode_11725,errcode_1174404,errcode_117471,errcode_11751,errcode_11818,errcode_118518,errcode_11864,errcode_11886,errcode_11906,errcode_11942,errcode_11959,errcode_12,errcode_120256,errcode_12026,errcode_12093,errcode_12099,errcode_12103,errcode_12120,errcode_12125,...,errcode_8252,errcode_8255,errcode_82561,errcode_8257,errcode_8258,errcode_8260,errcode_8263,errcode_82694,errcode_8271,errcode_8278,errcode_82799,errcode_83,errcode_8310,errcode_8322,errcode_8339,errcode_83468,errcode_8350,errcode_8355,errcode_8356,errcode_8372,errcode_8389,errcode_8391,errcode_8397,errcode_8398,errcode_84,errcode_8408,errcode_8414,errcode_8442,errcode_8453,errcode_8459,errcode_8460,errcode_8462,errcode_8473,errcode_8488,errcode_8494,errcode_85,errcode_8515,errcode_8522,errcode_8533,errcode_8543,errcode_8549,errcode_85885,errcode_8592,errcode_86,errcode_8606,errcode_8610,errcode_8637,errcode_8641,errcode_8645,errcode_8664,errcode_8673,errcode_8674,errcode_8684,errcode_8699,errcode_87,errcode_8719,errcode_8731,errcode_8733,errcode_8738,errcode_8740,errcode_87418,errcode_87427,errcode_8755,errcode_8760,errcode_8764,errcode_8771,errcode_8793,errcode_88,errcode_8811,errcode_8819,errcode_8840,errcode_8843,errcode_8

In [7]:
# y 만들기
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 

In [12]:
print(np.mean(auc_scores))

0.8150476866883988


In [13]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [15]:
X_test.head(1)

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,day1203,day1204,day1205,day1209,day1211,day1214,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-1010,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10020,errcode_10029,errcode_10030,errcode_10035,errcode_10043,errcode_10046,errcode_10072,errcode_101,errcode_10158,errcode_10165,errcode_10168,errcode_102,errcode_10217,errcode_10237,errcode_10274,errcode_103,errcode_103063,errcode_10330,errcode_10331,errcode_10334,errcode_10335,errcode_10361,errcode_104,errcode_10400,errcode_10475,errcode_10487,errcode_104942,errcode_105,errcode_10500,errcode_10512,errcode_10522,errcode_10523,errcode_10527,errcode_10553,errcode_10558,errcode_10563,errcode_10564,errcode_10567,errcode_10576,errcode_10580,errcode_10581,errcode_10585,errcode_10591,errcode_10598,errcode_10599,errcode_106,errcode_10609,errcode_10626,errcode_10638,errcode_10651,errcode_10653,errcode_10655,errcode_10659,errcode_10668,errcode_10675,errcode_10680,errcode_10683,errcode_10685,errcode_10686,errcode_10695,errcode_10698,errcode_107,errcode_10711,errcode_10717,errcode_10730,errcode_10731,errcode_10734,errcode_10736,errcode_10743,errcode_10751,errcode_10753,errcode_10763,errcode_10774,errcode_10779,errcode_10784,errcode_10785,errcode_10788,errcode_10790,errcode_108,errcode_10804,errcode_10805,errcode_10806,errcode_10812,errcode_10817,errcode_10822,errcode_10825,errcode_10829,errcode_10830,errcode_10843,errcode_10845,errcode_10848,errcode_10853,errcode_10857,errcode_10860,errcode_10867,errcode_10872,errcode_10874,errcode_108761,errcode_10881,errcode_10890,errcode_10894,errcode_10899,errcode_109,errcode_10911,errcode_10913,errcode_10915,errcode_10920,errcode_10923,errcode_10930,errcode_10935,errcode_10944,errcode_10963,errcode_10966,errcode_10979,errcode_10984,errcode_10985,errcode_109851,errcode_10987,errcode_10991,errcode_10992,errcode_10993,errcode_110,errcode_11002,errcode_11006,errcode_11010,errcode_11011,errcode_11023,errcode_11027,errcode_11037,errcode_11044,errcode_11049,errcode_11067,errcode_11077,errcode_11085,errcode_11090,errcode_11100,errcode_11105,errcode_11116,errcode_11124,errcode_11138,errcode_11160,errcode_11161422,errcode_11226,errcode_11246,errcode_112485,errcode_11308,errcode_11336,errcode_113979,errcode_11398,errcode_11416,...,errcode_8479,errcode_8492,errcode_8497,errcode_85,errcode_8507,errcode_8517,errcode_8540,errcode_8543,errcode_8549,errcode_8554,errcode_8558,errcode_8562,errcode_8564,errcode_8565,errcode_8571,errcode_8572,errcode_8577,errcode_8579,errcode_8583,errcode_8595,errcode_8599,errcode_86,errcode_8603,errcode_8605,errcode_8622,errcode_8630,errcode_8642,errcode_8646,errcode_8663,errcode_8667,errcode_8671,errcode_8682,errcode_8683,errcode_8699,errcode_87,errcode_8708,errcode_8715,errcode_8718,errcode_8720,errcode_8723,errcode_8726,errcode_8730,errcode_8731,errcode_8737,errcode_87429,errcode_8743,errcode_8745,errcode_8754,errcode_8758,errcode_87621,errcode_8763,errcode_8771,errcode_8773,errcode_8784,errcode_88,errcode_8802,errcode_8831,errcode_8834,errcode_8839,errcode_8843,errcode_8848,errcode_8849,errcode_8852,errcode_8855,errcode_8857,errcode_8868,errcode_8881,errcode_8889,errcode_8899,errcode_89,errcode_8913,errcode_8916,errcode_8924,errcode_8932,errcode_8941,errcode_8942,

In [16]:
X_train.to_csv("X_train_ej.csv", index = False)

In [17]:
X_test.to_csv("X_test_ej.csv", index = False)

In [4]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')

In [5]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_quality = pd.read_csv(PATH+'test_quality_data.csv')

# 함수

In [42]:
def make_df(train_err, quality, select=0):
    
    # select=0 : errday, quality, errtype, errcode, model, fwver
    # select=1 : errday, quality, errtype*errcode, model, fwver
    # select=2 : errday, quality, errtype*errcode*fwver, model    
        
    
    #train_err  = pd.read_csv(PATH+'train_err_data.csv')
    train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
    train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
    train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

    train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
    train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
    train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
    train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))
    
    
    
    ###err_day
    pivot_err_day = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
    df_errday = pd.DataFrame(pivot_err_day.to_records())

    df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

    for i in list(range(1,33)) :
            df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

   
    
    ###quality
    
    #quality = pd.read_csv(PATH+'train_quality_data.csv')
    quality.fillna(0, inplace = True) #quality_5변수에 NA값이 있음

    cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
              'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
    quality[cols] = quality[cols].replace({',':''}, regex= True) 
    quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

    minusQ = []
    for i in range(0,len(quality)): 
        if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
        #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
          minusQ.append(-1)
        else: minusQ.append(0)

    quality['minusQ'] = minusQ 


    def func(x):
        d = {}
        d['quality_0'] = x['quality_0'].max()
        d['quality_1'] = x['quality_1'].max()
        d['quality_2'] = x['quality_2'].max()
        d['quality_3'] = x['quality_3'].max()
        d['quality_4'] = x['quality_4'].max()
        d['quality_5'] = x['quality_5'].max()
        d['quality_6'] = x['quality_6'].max()
        d['quality_7'] = x['quality_7'].max()
        d['quality_8'] = x['quality_8'].max()
        d['quality_9'] = x['quality_9'].max()
        d['quality_10'] = x['quality_10'].max()
        d['quality_11'] = x['quality_11'].max()
        d['quality_12'] = x['quality_12'].max()
        d['minusQ'] = x['minusQ'].sum()
        
        return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                      'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

    quality5 = quality.groupby(['user_id','time']).apply(func)

    quality5 = quality5.groupby(['user_id']).sum()
    quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

  

    ###err_type
    pivot_err_type = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype = pd.DataFrame(pivot_err_type.to_records())
    df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 



    ###errcode
    pivot_errcode = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
    df_errcode = pd.DataFrame(pivot_errcode.to_records())

    df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))



    ### model
    pivot_model = pd.pivot_table(train_err, index = ['user_id'],columns = 'model_nm', values = ['time'],aggfunc='count',fill_value=0)
    df_model = pd.DataFrame(pivot_model.to_records())

    df_model = df_model.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time",""))

    

    ###fwver
    pivot_fwver = pd.pivot_table(train_err, index = ['user_id'],columns = 'fwver', values = ['time'],aggfunc='count',fill_value=0)
    df_fwver = pd.DataFrame(pivot_fwver.to_records())

    df_fwver = df_fwver.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","fwver"))
    
    
    
    ###errtype*errcode
    temp1 = train_err
    temp1['errtype_code'] = temp1['errtype'].astype(str)+'_'+temp1['errcode']
    
    pivot_errtype_code = pd.pivot_table(temp1, index = ['user_id'],columns = 'errtype_code', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype_code = pd.DataFrame(pivot_errtype_code.to_records())

    df_errtype_code = df_errtype_code.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype_code"))
        
    
    ###errtype*errcode*fwver
    temp1['errtype_code_fwver'] = temp1['errtype'].astype(str)+'_'+temp1['errcode']+'_'+temp1['fwver']
    
    pivot_errtype_code_fwver = pd.pivot_table(temp1, index = ['user_id'],columns = 'errtype_code_fwver', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype_code_fwver = pd.DataFrame(pivot_errtype_code_fwver.to_records())

    df_errtype_code_fwver = df_errtype_code_fwver.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype_code_fwver"))
            
    
    
    
    # select=0 : errday, quality, errtype, errcode, model, fwver
    # select=1 : errday, quality, errtype*errcode, model, fwver
    # select=2 : errday, quality, errtype*errcode*fwver, model    
    
    
    
    
    
    ####merge
    
    if select==0:
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errcode, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        df_result = pd.merge(df_result, df_fwver, on='user_id', how='left')
        
    
    elif select==1:
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype_code, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        df_result = pd.merge(df_result, df_fwver, on='user_id', how='left')
        
    else :
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype_code_fwver, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        

    df_result.fillna(0, inplace = True)

    return df_result

In [29]:
X_train0 = make_df(train_err, train_quality, select=0)

In [30]:
display(X_train0.head())
print(X_train0.columns)

,user_id,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10005,errcode_10018,errcode_10043,errcode_10073,errcode_10080,errcode_101,errcode_10101,errcode_10149,errcode_10153,errcode_10155,errcode_10164,errcode_10190,errcode_102,errcode_10222,errcode_10226,errcode_10240,errcode_10247,errcode_10263,errcode_10267,errcode_10274,errcode_102789,errcode_10280,errcode_103,errcode_10305,errcode_10309,errcode_103400,errcode_10350,errcode_10377,errcode_10385,errcode_103924,errcode_104,errcode_10406,errcode_10412,errcode_10420,errcode_10433,errcode_105,errcode_10518,errcode_10552,errcode_10554,errcode_10557,errcode_10566,errcode_10568,errcode_105690,errcode_10573,errcode_10584,errcode_106,errcode_10612,errcode_10630,errcode_10670,errcode_10674,errcode_10676,errcode_10684,errcode_10689,errcode_10692,errcode_107,errcode_10715,errcode_10720,errcode_10721,errcode_10725,errcode_10732,errcode_10743,errcode_10749,errcode_10752,errcode_10760,errcode_10768,errcode_10777,errcode_10782,errcode_10792,errcode_108,errcode_10803,errcode_10807,errcode_10812,errcode_108133,errcode_10814,errcode_10819,errcode_10827,errcode_10830,errcode_10833,errcode_10838,errcode_10839,errcode_10841,errcode_10851,errcode_10852,errcode_10861,errcode_10881,errcode_10894,errcode_10898,errcode_1089908,errcode_109,errcode_10902,errcode_10907,errcode_10908,errcode_10933,errcode_10934,errcode_10938,errcode_10945,errcode_10949,errcode_10955,errcode_10961,errcode_10972,errcode_10980,errcode_10995,errcode_10997,errcode_110,errcode_11007,errcode_11014,errcode_11015,errcode_11033,errcode_11043,errcode_11063,errcode_11068,errcode_11070,errcode_11078,errcode_11087,errcode_11088,errcode_11094,errcode_11100,errcode_11108,errcode_11113,errcode_11146,errcode_11151,errcode_11197,errcode_11251,errcode_11259,errcode_112910,errcode_11295,errcode_113066,errcode_11324,errcode_11355,errcode_11396,errcode_11399,errcode_11434,errcode_11516,errcode_11534,errcode_11650,errcode_1166733,errcode_11674,errcode_11717,errcode_11725,errcode_1174404,errcode_117471,errcode_11751,errcode_11818,errcode_118518,errcode_11864,errcode_11886,errcode_11906,errcode_11942,errcode_11959,errcode_12,errcode_120256,errcode_12026,errcode_12093,errcode_12099,errcode_12103,errcode_12120,errcode_12125,...,errcode_8637,errcode_8641,errcode_8645,errcode_8664,errcode_8673,errcode_8674,errcode_8684,errcode_8699,errcode_87,errcode_8719,errcode_8731,errcode_8733,errcode_8738,errcode_8740,errcode_87418,errcode_87427,errcode_8755,errcode_8760,errcode_8764,errcode_8771,errcode_8793,errcode_88,errcode_8811,errcode_8819,errcode_8840,errcode_8843,errcode_8851,errcode_8855,errcode_89,errcode_8915,errcode_8918,errcode_892446,errcode_8928,errcode_8931,errcode_8937,errcode_8940,errcode_8944,errcode_8974,errcode_90,errcode_9028,errcode_9030,errcode_9032,errcode_9033,errcode_9044,errcode_9069,errcode_9088,errcode_91,errcode_9131,errcode_9181,errcode_9185,errcode_9197,errcode_9199,errcode_92,errcode_9214,errcode_9229,errcode_9232,errcode_9262,errcode_92862,errcode_9288,errcode_93,errcode_9301,errcode_9311,errcode_93129,errcode_9314,errcode_9323,errcode_9334,errcode_9354,errcode_9364,errcode_9395,errcode_9398,errcode_94,errcode_9402,errcode_9462,e

Index(['user_id', 'day1031', 'day1101', 'day1102', 'day1103', 'day1104',
       'day1105', 'day1106', 'day1107', 'day1108',
       ...
       'fwver05.15.2092', 'fwver05.15.2114', 'fwver05.15.2120',
       'fwver05.15.2122', 'fwver05.15.2138', 'fwver05.15.3104',
       'fwver05.66.3237', 'fwver05.66.3571', 'fwver10', 'fwver8.5.3'],
      dtype='object', length=2938)


In [37]:
X_train0.shape

(15000, 2938)

In [34]:
X_train1 = make_df(train_err, train_quality, select=1)

In [35]:
display(X_train1.head())
print(X_train1.columns)

,user_id,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errtype_code10_1,errtype_code11_1,errtype_code12_1,errtype_code13_1,errtype_code14_1,errtype_code14_13,errtype_code14_14,errtype_code15_1,errtype_code16_1,errtype_code17_1,errtype_code17_12,errtype_code17_13,errtype_code17_14,errtype_code17_21,errtype_code18_1,errtype_code19_1,errtype_code1_0,errtype_code1_P-41001,errtype_code1_P-41007,errtype_code1_P-41007,errtype_code1_P-41011,errtype_code1_P-41011,errtype_code1_P-44010,errtype_code1_P-44010,errtype_code20_1,errtype_code21_1,errtype_code22_1,errtype_code23_UNKNOWN,errtype_code23_active,errtype_code23_connectionfailforLMPresponsetimout,errtype_code23_connectionfailtoestablish,errtype_code23_connectionerrtype_codeout,errtype_code23_connectionterminatedbylocalhost,errtype_code23_standby,errtype_code23_terminatebypeeruser,errtype_code24_1,errtype_code25_1,errtype_code25_2,errtype_code25_L2CAPconnectioncancelled,errtype_code25_UNKNOWN,errtype_code25_connectionfailtoestablish,errtype_code25_connectionerrtype_codeout,errtype_code25_connectionterminatedbylocalhost,errtype_code25_scanningerrtype_codeout,errtype_code25_terminatebypeeruser,errtype_code26_1,errtype_code27_1,errtype_code28_1,errtype_code2_0,errtype_code2_1,errtype_code30_0,errtype_code30_1,errtype_code30_2,errtype_code30_3,errtype_code30_4,errtype_code31_0,errtype_code31_1,errtype_code32_-269,errtype_code32_-270,errtype_code32_100,errtype_code32_101,errtype_code32_102,errtype_code32_103,errtype_code32_104,errtype_code32_105,errtype_code32_106,errtype_code32_107,errtype_code32_108,errtype_code32_109,errtype_code32_110,errtype_code32_37,errtype_code32_38,errtype_code32_41,errtype_code32_42,errtype_code32_46,errtype_code32_47,errtype_code32_62,errtype_code32_65,errtype_code32_66,errtype_code32_67,errtype_code32_68,errtype_code32_69,errtype_code32_70,errtype_code32_71,errtype_code32_72,errtype_code32_73,errtype_code32_74,errtype_code32_75,errtype_code32_76,errtype_code32_77,errtype_code32_78,errtype_code32_79,errtype_code32_80,errtype_code32_81,errtype_code32_82,errtype_code32_83,errtype_code32_84,errtype_code32_85,errtype_code32_86,errtype_code32_87,errtype_code32_88,errtype_code32_89,errtype_code32_90,errtype_code32_91,errtype_code32_92,errtype_code32_93,errtype_code32_94,errtype_code32_95,errtype_code32_96,errtype_code32_97,errtype_code32_98,errtype_code32_99,errtype_code33_1,errtype_code33_2,errtype_code33_3,errtype_code34_1,errtype_code34_2,errtype_code34_3,errtype_code34_4,errtype_code34_5,errtype_code34_6,errtype_code35_1,errtype_code36_8.0,errtype_code37_0,errtype_code37_1,errtype_code38_10005,errtype_code38_10018,errtype_code38_10043,errtype_code38_10073,errtype_code38_10080,errtype_code38_10101,errtype_code38_10149,errtype_code38_10153,errtype_code38_10155,errtype_code38_10164,errtype_code38_10190,errtype_code38_10222,errtype_code38_10226,errtype_code38_10240,errtype_code38_10247,errtype_code38_10263,errtype_code38_10267,errtype_code38_10274,errtype_code38_102789,errtype_code38_10280,errtype_code38_10305,errtype_code38_10309,errtype_code38_103400,errtype_code38_10350,errtype_code38_10377,errtype_code38_10385,errtype_code38_103924,errtype_code38_10406,errtype_code38_10412,errtype_code38_10420,errtype_code38_10433,errtype_code38_10518,errtype_code38_10552,errtype_code38_10554,errtype_code38_10557,errtype_code38_10566,errtype_code38_10568,errtype_code38_105690,errtype_code38_10573,errtype_code38_10584,errtype_code38_10612,errtype_code38_10630,errtype_code38_10670,errtype_code38_10674,errtype_code38_10676,errtype_code38_10684,errtype_code38_10689,errtype_code38_10692,errtype_code38_10715,errtype_code38_10720,errtype_code38_10721,errt

Index(['user_id', 'day1031', 'day1101', 'day1102', 'day1103', 'day1104',
       'day1105', 'day1106', 'day1107', 'day1108',
       ...
       'fwver05.15.2092', 'fwver05.15.2114', 'fwver05.15.2120',
       'fwver05.15.2122', 'fwver05.15.2138', 'fwver05.15.3104',
       'fwver05.66.3237', 'fwver05.66.3571', 'fwver10', 'fwver8.5.3'],
      dtype='object', length=2962)


In [38]:
X_train1.shape

(15000, 2962)

In [43]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
X_train2 = make_df(train_err, train_quality, select=2)

In [44]:
display(X_train2.head())
print(X_train2.columns)
print(X_train2.shape)

,user_id,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errtype_code_fwver10_1_03.11.1167,errtype_code_fwver10_1_04.16.3345,errtype_code_fwver10_1_04.16.3553,errtype_code_fwver10_1_04.16.3569,errtype_code_fwver10_1_04.16.3571,errtype_code_fwver10_1_04.22.1442,errtype_code_fwver10_1_04.22.1684,errtype_code_fwver10_1_04.22.1750,errtype_code_fwver10_1_04.22.1778,errtype_code_fwver10_1_04.33.1149,errtype_code_fwver10_1_04.33.1185,errtype_code_fwver10_1_04.33.1261,errtype_code_fwver10_1_04.73.2237,errtype_code_fwver10_1_04.82.1684,errtype_code_fwver10_1_04.82.1730,errtype_code_fwver10_1_04.82.1778,errtype_code_fwver10_1_05.15.2090,errtype_code_fwver10_1_05.15.2120,errtype_code_fwver10_1_05.15.2138,errtype_code_fwver10_1_05.15.3104,errtype_code_fwver10_1_05.66.3237,errtype_code_fwver10_1_05.66.3571,errtype_code_fwver11_1_03.11.1167,errtype_code_fwver11_1_04.16.2641,errtype_code_fwver11_1_04.16.3345,errtype_code_fwver11_1_04.16.3439,errtype_code_fwver11_1_04.16.3553,errtype_code_fwver11_1_04.16.3569,errtype_code_fwver11_1_04.16.3571,errtype_code_fwver11_1_04.22.1442,errtype_code_fwver11_1_04.22.1656,errtype_code_fwver11_1_04.22.1684,errtype_code_fwver11_1_04.22.1750,errtype_code_fwver11_1_04.22.1778,errtype_code_fwver11_1_04.33.1095,errtype_code_fwver11_1_04.33.1125,errtype_code_fwver11_1_04.33.1149,errtype_code_fwver11_1_04.33.1171,errtype_code_fwver11_1_04.33.1185,errtype_code_fwver11_1_04.33.1261,errtype_code_fwver11_1_04.73.2237,errtype_code_fwver11_1_04.73.2571,errtype_code_fwver11_1_04.82.1684,errtype_code_fwver11_1_04.82.1730,errtype_code_fwver11_1_04.82.1778,errtype_code_fwver11_1_05.15.2090,errtype_code_fwver11_1_05.15.2092,errtype_code_fwver11_1_05.15.2120,errtype_code_fwver11_1_05.15.2122,errtype_code_fwver11_1_05.15.2138,errtype_code_fwver11_1_05.15.3104,errtype_code_fwver11_1_05.66.3237,errtype_code_fwver11_1_05.66.3571,errtype_code_fwver12_1_03.11.1149,errtype_code_fwver12_1_03.11.1167,errtype_code_fwver12_1_04.16.2641,errtype_code_fwver12_1_04.16.3345,errtype_code_fwver12_1_04.16.3439,errtype_code_fwver12_1_04.16.3553,errtype_code_fwver12_1_04.16.3569,errtype_code_fwver12_1_04.16.3571,errtype_code_fwver12_1_04.22.1442,errtype_code_fwver12_1_04.22.1656,errtype_code_fwver12_1_04.22.1666,errtype_code_fwver12_1_04.22.1684,errtype_code_fwver12_1_04.22.1750,errtype_code_fwver12_1_04.22.1778,errtype_code_fwver12_1_04.33.1095,errtype_code_fwver12_1_04.33.1125,errtype_code_fwver12_1_04.33.1149,errtype_code_fwver12_1_04.33.1171,errtype_code_fwver12_1_04.33.1185,errtype_code_fwver12_1_04.33.1261,errtype_code_fwver12_1_04.73.2237,errtype_code_fwver12_1_04.73.2571,errtype_code_fwver12_1_04.82.1684,errtype_code_fwver12_1_04.82.1730,errtype_code_fwver12_1_04.82.1778,errtype_code_fwver12_1_05.15.2090,errtype_code_fwver12_1_05.15.2092,errtype_code_fwver12_1_05.15.2120,errtype_code_fwver12_1_05.15.2122,errtype_code_fwver12_1_05.15.2138,errtype_code_fwver12_1_05.15.3104,errtype_code_fwver12_1_05.66.3237,errtype_code_fwver12_1_05.66.3571,errtype_code_fwver13_1_03.11.1167,errtype_code_fwver13_1_04.16.2641,errtype_code_fwver13_1_04.16.3553,errtype_code_fwver13_1_04.16.3571,errtype_code_fwver13_1_04.22.1442,errtype_code_fwver13_1_04.22.1656,errtype_code_fwver13_1_04.22.1666,errtype_code_fwver13_1_04.22.1684,errtype_code_fwver13_1_04.22.1750,errtype_code_fwver13_1_04.22.1778,errtype_code_fwver13_1_04.33.1095,errtype_code_fwver13_1_04.33.1149,errtype_code_fwver13_1_04.33.1185,errtype_code_fwver13_1_04.33.1261,errtype_code_fwver13_1_04.73.2237,errtype_code_fwver13_1_04.73.2571,errtype_code_fwver13_1_04.82.1684,errtype_code_fwver13_1_04.82.1730,errtype_code_fwver13_1_04.82.1778,errtype_code_fwver13_1_05.15.2092,err

Index(['user_id', 'day1031', 'day1101', 'day1102', 'day1103', 'day1104',
       'day1105', 'day1106', 'day1107', 'day1108',
       ...
       'errtype_code_fwver9_V-21008_04.33.1261', 'model_0', 'model_1',
       'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7',
       'model_8'],
      dtype='object', length=5401)
(15000, 5401)


In [45]:
X_train0.to_csv("X_train0.csv", index = False)
X_train1.to_csv("X_train1.csv", index = False)
X_train2.to_csv("X_train2.csv", index = False)